<h1>Korean Text classification with KoBERT</h1>

## **Summary of the project**

In Korea, even though there are many research conducted being conducted on voice phishing, it remains a real case problem that technology such as artificial intelligence can tackle. Through a previous project conducted, we created a dataset containing phone call conversation transcripts and general conversation text data. This voice phishing dataset has two different class which are **voice phishing** (represented as "1") and **non-voice phishing** (represented as "0").

Using this dataset with state-of-the-art (SOTA) pre-trained word embedding [KoBERT](https://github.com/SKTBrain/KoBERT), we will perform NLP task such as text classification to build binary classification models.

## **Aim of the project**
In this project, we aim to build binary classification models capable to determine whether the inputted Korean conversation text is voice phishing ("1") or non-voice phishing ("0") related text.

The API used are Tensorflow for BERT model and Pytorch for KoBERT model.

## **Desired outputs of the project**
From the trained models, we expect to achieve great classification performance on this voice phishing dataset such as the model tells us if a conversation is harmful or not harmful.
At the end of this project, we will look at the accuracy of the model on the test set.

# Training the binary classification model with KoBERT

In [4]:
!nvidia-smi

'nvidia-smi'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


## Installing the common needed packages

In [5]:
# dowload and install KoBERT as a python package
# this commande will install the requirted package at the same time
  # gluonnlp >= 0.6.0
  # mxnet >= 1.4.0
  # onnxruntime >= 0.3.0
  # sentencepiece >= 0.1.6
  # torch >= 1.7.0
  # transformers >= 4.8.1

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to c:\users\hye42\appdata\local\temp\pip-req-build-_sr58v90
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit fcd729f2f4b37858f333597c0782388ada51eb5f
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/9f/1c/a17fb513aeb684fb83bef5f395910f53103ab30308bbdd77fd66d6698c46/accelerate-1.9.0-py3-none-any.whl.metadata
  Using cached accelerate-1.9.0-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/eb/62/eb8157afb21bd229c864521c1ab4fa8e9b4f1b06bafdd8c4668a7a31b5d

  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' 'C:\Users\hye42\AppData\Local\Temp\pip-req-build-_sr58v90'
  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [27 lines of output]
  Running from numpy source directory.
  <string>:394: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
  C:\Users\hye42\AppData\Local\Temp\pip-install-vfe4ptdt\numpy_c2c25a7202384645a1163a216c1a5eb7\numpy\distutils\misc_util.py:476: SyntaxWarning: "is" with a literal. Did you mean "=="?
    return is_string(s) and ('*' in s or '?' is s)
  Traceback (most recent call last):
    File "C:\Users\hye42\Korean_Voice_Phishing_Detection\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Users\hye42\Korean_Voice_Phishing_Detection\venv\Lib\site-packages\pi

## Import all the needed libraries

In [6]:
## importing the required packages
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'gluonnlp'

In [ ]:
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
# from keras.models import Sequential

In [ ]:
## importing KoBERT functions
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
## import transformers functions
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
## Configure the GPU  device
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

## Importing the dataset

In [ ]:
"""
Since we are using Colab, we will provide a test to check if environment is 
colab or not so that the data can also be imported in case this jupyter file is 
ran on local machine and not on colab
"""

if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  ## mount the google drive
  from google.colab import drive
  drive.mount('drive')
  # move to the directory where dataset is saved
  %cd drive/My\ Drive/Colab\ Notebooks/
else:
  print('Not running on CoLab')

In [ ]:
# import the dataset
dataset = pd.read_csv('KorCCViD_v1.3_fullcleansed.csv').sample(frac=1.0)
dataset.sample(n=15)

## Data transformation and splitting

In [ ]:
## transform our train set and test set into tsv file to usedd into KoBERT
# train_tsv = nlp.data.TSVDataset('KorCCViD_v1.3_fullcleansed.csv')
# train_tsv = nlp.data.TSVDataset('KorCCViD_v1.3_fullcleansed.csv')

dataset_tsv = []
for text, label in zip(dataset['Transcript'], dataset['Label']):
    data = []
    data.append(text)
    data.append(str(label))

    dataset_tsv.append(data)

In [ ]:
dataset_tsv[:5]

In [ ]:
# Split the data into train set and test set

# train_set, test_set = train_test_split(dataset_tsv, 
#                                test_size=0.3, 
#                                random_state=42, 
#                                shuffle=True)
# print(f"Numbers of train instances by class: {len(train_set)}")
# print(f"Numbers of test instances by class: {len(test_set)}")

train_set, val_set = train_test_split(dataset_tsv, 
                               test_size=0.2, 
                               random_state=42, 
                               shuffle=True)

# train_set, val_set = train_test_split(train_set, 
#                                test_size=0.2, 
#                                random_state=42, 
#                                shuffle=True)
print(f"Numbers of train instances by class: {len(train_set)}")
print(f"Numbers of val instances by class: {len(val_set)}")
# print(f"Numbers of test instances by class: {len(test_set)}")



## Prepare the data as input for the KoBERT model
According tot he documentation the class BERTDataset is to be used to perform in the background the following tasks.
- Tokenization
- Numericalization (encoding string to integer)
- Padding
- etc



In [ ]:
# Definition of BERTDataset class (mandatory)
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting the hyperparameters
max_len = 64 # The maximum sequence length that this model might ever be used with. 
             # Typically set this to something large just in case (e.g., 512 or 1024 or 2048).
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10   # only parameter changed from 5 to 10 compared to the documentation
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5  # 4e-5

In [ ]:
# Perform the prearation task of the data using class defined above
bertmodel, vocab = get_pytorch_kobert_model() # calling the bert model and the vocabulary

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

train_set = BERTDataset(train_set, 0, 1, tok, max_len, True, False)
val_set = BERTDataset(val_set, 0, 1, tok, max_len, True, False)
#test_set = BERTDataset(test_set, 0, 1, tok, max_len, True, False)

In [ ]:
tokenizer

In [ ]:
tok

In [ ]:
# verifying the transformation
train_set[1]

In [ ]:
# creating torch-type datasets
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=5)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, num_workers=5)
#test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)

## Creation of the KoBERT learing model

In [ ]:
# This class is from the GitHub repository and the documentation
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   # since we are in binary classification we set the value 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
import torch
# torch.cuda.empty_cache()

In [ ]:
# creation of the model
model = BERTClassifier(bertmodel,  dr_rate=0.4).to(device)

In [ ]:
%%time
print(model)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# configuration f the optimizer and loss function
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
# define the function to compute the accury of the model
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# model.summary()

In [ ]:
def get_metrics(pred, label, threshold=0.5):
    pred = (pred > threshold).astype('float32')
    tp = ((pred == 1) & (label == 1)).sum()
    fp = ((pred == 1) & (label == 0)).sum()
    fn = ((pred == 0) & (label == 1)).sum()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * recall * precision / (precision + recall)
    
    return {
        'recall': recall,
        'precision': precision,
        'f1': f1
    }

## Training the KoBERT model


In [ ]:
%%time
from time import time
from timeit import default_timer as timer

# Training code from the github library
start_time = time()

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    # Training of the model with the train set
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    preds = []
    labels = []
    # evaluation of the model train on the test set
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        labe2 = label.cpu()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
        
        pred = out.detach()
        pred = F.softmax(pred)
        pred = pred[:, 1].cpu().numpy().tolist()
        preds += pred
        labels += label.cpu().numpy().tolist()
        
    preds = np.array(preds)
    labels = np.array(labels)
    metrics = get_metrics(preds, labels)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # print('ACCURACY 2 = ', accuracy_score(out, label))
    print('Metrics: ', metrics)

run_time = time() - start_time

In [ ]:
run_time
#224.96386766433716
#0.9947

In [ ]:
preds = []
labels = []
test_acc = 0.0
# evaluation of the model train on the test set
model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    labe2 = label.cpu()
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)

    pred = out.detach()
    pred = F.softmax(pred)
    pred = pred[:, 1].cpu().numpy().tolist()
    preds += pred
    labels += label.cpu().numpy().tolist()
    
preds = np.array(preds)
labels = np.array(labels)
metrics = get_metrics(preds, labels)
print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
# print('ACCURACY 2 = ', accuracy_score(out, label))
print('Metrics: ', metrics)

<h2>Model Training result</h2>

From the previous training result, we can see that our KoBERT binary classification model reached **99.68%** of accuracy on the test set and **100**%  of accuracy on the train set.